In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans as SklearnKMeans
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score, normalized_mutual_info_score
)
from sklearn.decomposition import PCA
import time
import warnings

In [ ]:
warnings.filterwarnings('ignore')

class KMeansFromScratch:
    """
    K-Means Clustering implemented from scratch
    """

    def __init__(self, n_clusters=3, max_iters=300, tolerance=1e-4, random_state=42):
        """
        Parameters:
        -----------
        n_clusters : int
            Number of clusters
        max_iters : int
            Maximum number of iterations
        tolerance : float
            Convergence threshold
        random_state : int
            Random seed for reproducibility
        """
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        # self.tolerance = tolerance
        self.random_state = random_state
        self.centroids = None
        self.labels_ = None
        self.inertia_ = None
        self.n_iter_ = 0

    def _initialize_centroids(self, X):
        """Initialize centroids using random selection"""
        np.random.seed(self.random_state)
        random_indices = np.random.choice(X.shape[0], self.n_clusters, replace=False)
        centroids = X[random_indices]
        return centroids

    def _calculate_distance(self, X, centroids):
        """Calculate Euclidean distance between points and centroids"""
        distances = np.zeros((X.shape[0], self.n_clusters))

        for k in range(self.n_clusters):
            # Euclidean distance: sqrt(sum((x - centroid)^2))
            distances[:, k] = np.sqrt(np.sum((X - centroids[k])**2, axis=1))

        return distances

    def _assign_clusters(self, distances):
        """Assign each point to nearest centroid"""
        return np.argmin(distances, axis=1)

    def _update_centroids(self, X, labels):
        """Update centroids as mean of assigned points"""
        centroids = np.zeros((self.n_clusters, X.shape[1]))

        for k in range(self.n_clusters):
            # Get all points assigned to cluster k
            cluster_points = X[labels == k]

            if len(cluster_points) > 0:
                # Calculate mean of cluster points
                centroids[k] = np.mean(cluster_points, axis=0)
            else:
                # If cluster is empty, reinitialize randomly
                centroids[k] = X[np.random.choice(X.shape[0])]

        return centroids

    def _calculate_inertia(self, X, labels, centroids):
        """Calculate within-cluster sum of squares (inertia)"""
        inertia = 0
        for k in range(self.n_clusters):
            cluster_points = X[labels == k]
            if len(cluster_points) > 0:
                inertia += np.sum((cluster_points - centroids[k])**2)
        return inertia

    def fit(self, X):
        """
        Fit K-Means clustering

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        """
        # Convert to numpy array
        X = np.array(X)

        # Initialize centroids
        self.centroids = self._initialize_centroids(X)

        # Iterative optimization
        for iteration in range(self.max_iters):
            # Store old centroids for convergence check
            old_centroids = self.centroids.copy()

            # Step 1: Calculate distances to all centroids
            distances = self._calculate_distance(X, self.centroids)

            # Step 2: Assign points to nearest centroid
            self.labels_ = self._assign_clusters(distances)

            # Step 3: Update centroids
            self.centroids = self._update_centroids(X, self.labels_)

            # Check for convergence
            centroid_shift = np.sqrt(np.sum((self.centroids - old_centroids)**2))

            if centroid_shift < self.tolerance:
                self.n_iter_ = iteration + 1
                break
        else:
            self.n_iter_ = self.max_iters

        # Calculate final inertia
        self.inertia_ = self._calculate_inertia(X, self.labels_, self.centroids)

        return self

    def predict(self, X):
        """
        Predict cluster labels for new data

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            New data

        Returns:
        --------
        labels : array, shape (n_samples,)
            Cluster labels
        """
        X = np.array(X)
        distances = self._calculate_distance(X, self.centroids)
        return self._assign_clusters(distances)

    def fit_predict(self, X):
        """Fit and predict in one step"""
        self.fit(X)
        return self.labels_


class KMeansComparison:
    """
    Compare custom K-Means with sklearn implementation
    """

    def __init__(self, X, true_labels=None):
        """
        Parameters:
        -----------
        X : array-like
            Data to cluster
        true_labels : array-like, optional
            True labels if available (for evaluation)
        """
        self.X = np.array(X)
        self.true_labels = true_labels
        self.results = {}

    def run_comparison(self, k_values=[2, 3, 4, 5, 6], random_state=42):
        """
        Run both implementations for different k values

        Parameters:
        -----------
        k_values : list
            List of k values to test
        random_state : int
            Random seed
        """
        print("="*70)
        print("K-MEANS COMPARISON: FROM SCRATCH VS SKLEARN")
        print("="*70)

        for k in k_values:
            print(f"\n{'='*70}")
            print(f"Testing with K = {k}")
            print('='*70)

            # Custom Implementation
            print("\n[1] Custom K-Means (From Scratch)")
            print("-" * 50)
            start_time = time.time()

            custom_kmeans = KMeansFromScratch(
                n_clusters=k,
                max_iters=300,
                random_state=random_state
            )
            custom_labels = custom_kmeans.fit_predict(self.X)

            custom_time = time.time() - start_time
            custom_inertia = custom_kmeans.inertia_
            custom_iterations = custom_kmeans.n_iter_

            print(f"✓ Time taken: {custom_time:.4f} seconds")
            print(f"✓ Iterations: {custom_iterations}")
            print(f"✓ Inertia: {custom_inertia:.2f}")

            # Sklearn Implementation
            print("\n[2] Sklearn K-Means")
            print("-" * 50)
            start_time = time.time()

            sklearn_kmeans = SklearnKMeans(
                n_clusters=k,
                max_iter=300,
                random_state=random_state,
                n_init=10  # sklearn runs 10 times by default
            )
            sklearn_labels = sklearn_kmeans.fit_predict(self.X)

            sklearn_time = time.time() - start_time
            sklearn_inertia = sklearn_kmeans.inertia_
            sklearn_iterations = sklearn_kmeans.n_iter_

            print(f"✓ Time taken: {sklearn_time:.4f} seconds")
            print(f"✓ Iterations: {sklearn_iterations}")
            print(f"✓ Inertia: {sklearn_inertia:.2f}")

            # Evaluation Metrics
            print("\n[3] Clustering Quality Metrics")
            print("-" * 50)

            """
            | Metric            | Measures                               | Best Value | Sensitive To            |
            | ----------------- | -------------------------------------- | ---------- | ----------------------- |
            | Silhouette        | Compactness + separation (point-level) | Higher     | Overlapping clusters    |
            | Davies–Bouldin    | Worst-case cluster similarity          | Lower      | Noisy / spread clusters |
            | Calinski–Harabasz | Global variance ratio                  | Higher     | Number of clusters      |

            """

            # Silhouette Score (higher is better, range: -1 to 1)
            custom_silhouette = silhouette_score(self.X, custom_labels)
            sklearn_silhouette = silhouette_score(self.X, sklearn_labels)

            # Davies-Bouldin Index (lower is better)
            custom_db = davies_bouldin_score(self.X, custom_labels)
            sklearn_db = davies_bouldin_score(self.X, sklearn_labels)

            # Calinski-Harabasz Index (higher is better)
            custom_ch = calinski_harabasz_score(self.X, custom_labels)
            sklearn_ch = calinski_harabasz_score(self.X, sklearn_labels)

            print(f"Silhouette Score:")
            print(f"  Custom:  {custom_silhouette:.4f}")
            print(f"  Sklearn: {sklearn_silhouette:.4f}")
            print(f"\nDavies-Bouldin Index (lower is better):")
            print(f"  Custom:  {custom_db:.4f}")
            print(f"  Sklearn: {sklearn_db:.4f}")
            print(f"\nCalinski-Harabasz Index (higher is better):")
            print(f"  Custom:  {custom_ch:.2f}")
            print(f"  Sklearn: {sklearn_ch:.2f}")

            # If true labels available, calculate external validation
            if self.true_labels is not None:
                custom_ari = adjusted_rand_score(self.true_labels, custom_labels)
                sklearn_ari = adjusted_rand_score(self.true_labels, sklearn_labels)

                custom_nmi = normalized_mutual_info_score(self.true_labels, custom_labels)
                sklearn_nmi = normalized_mutual_info_score(self.true_labels, sklearn_labels)

                print(f"\nAdjusted Rand Index (vs true labels):")
                print(f"  Custom:  {custom_ari:.4f}")
                print(f"  Sklearn: {sklearn_ari:.4f}")
                print(f"\nNormalized Mutual Info (vs true labels):")
                print(f"  Custom:  {custom_nmi:.4f}")
                print(f"  Sklearn: {sklearn_nmi:.4f}")
            else:
                custom_ari = sklearn_ari = None
                custom_nmi = sklearn_nmi = None

            # Store results
            self.results[k] = {
                'custom': {
                    'model': custom_kmeans,
                    'labels': custom_labels,
                    'time': custom_time,
                    'inertia': custom_inertia,
                    'iterations': custom_iterations,
                    'silhouette': custom_silhouette,
                    'davies_bouldin': custom_db,
                    'calinski_harabasz': custom_ch,
                    'ari': custom_ari,
                    'nmi': custom_nmi
                },
                'sklearn': {
                    'model': sklearn_kmeans,
                    'labels': sklearn_labels,
                    'time': sklearn_time,
                    'inertia': sklearn_inertia,
                    'iterations': sklearn_iterations,
                    'silhouette': sklearn_silhouette,
                    'davies_bouldin': sklearn_db,
                    'calinski_harabasz': sklearn_ch,
                    'ari': sklearn_ari,
                    'nmi': sklearn_nmi
                }
            }

        print("\n" + "="*70)
        print("✓ COMPARISON COMPLETE!")
        print("="*70)

    def plot_comparison(self):
        """Generate comprehensive comparison visualizations"""

        print("\n" + "="*70)
        print("GENERATING COMPARISON VISUALIZATIONS")
        print("="*70)

        k_values = list(self.results.keys())

        # Prepare data for plotting
        custom_metrics = {
            'inertia': [],
            'silhouette': [],
            'davies_bouldin': [],
            'time': [],
            'iterations': []
        }

        sklearn_metrics = {
            'inertia': [],
            'silhouette': [],
            'davies_bouldin': [],
            'time': [],
            'iterations': []
        }

        for k in k_values:
            custom_metrics['inertia'].append(self.results[k]['custom']['inertia'])
            custom_metrics['silhouette'].append(self.results[k]['custom']['silhouette'])
            custom_metrics['davies_bouldin'].append(self.results[k]['custom']['davies_bouldin'])
            custom_metrics['time'].append(self.results[k]['custom']['time'])
            custom_metrics['iterations'].append(self.results[k]['custom']['iterations'])

            sklearn_metrics['inertia'].append(self.results[k]['sklearn']['inertia'])
            sklearn_metrics['silhouette'].append(self.results[k]['sklearn']['silhouette'])
            sklearn_metrics['davies_bouldin'].append(self.results[k]['sklearn']['davies_bouldin'])
            sklearn_metrics['time'].append(self.results[k]['sklearn']['time'])
            sklearn_metrics['iterations'].append(self.results[k]['sklearn']['iterations'])

        # Create subplots
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        fig.suptitle('K-Means Comparison: Custom vs Sklearn', fontsize=16, fontweight='bold')

        # 1. Elbow Plot (Inertia)
        axes[0, 0].plot(k_values, custom_metrics['inertia'], 'o-', label='Custom', linewidth=2, markersize=8)
        axes[0, 0].plot(k_values, sklearn_metrics['inertia'], 's-', label='Sklearn', linewidth=2, markersize=8)
        axes[0, 0].set_xlabel('Number of Clusters (K)', fontsize=10)
        axes[0, 0].set_ylabel('Inertia (Within-cluster SS)', fontsize=10)
        axes[0, 0].set_title('Elbow Method', fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(alpha=0.3)

        # 2. Silhouette Score
        axes[0, 1].plot(k_values, custom_metrics['silhouette'], 'o-', label='Custom', linewidth=2, markersize=8)
        axes[0, 1].plot(k_values, sklearn_metrics['silhouette'], 's-', label='Sklearn', linewidth=2, markersize=8)
        axes[0, 1].set_xlabel('Number of Clusters (K)', fontsize=10)
        axes[0, 1].set_ylabel('Silhouette Score', fontsize=10)
        axes[0, 1].set_title('Silhouette Score (Higher is Better)', fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(alpha=0.3)

        # 3. Davies-Bouldin Index
        axes[0, 2].plot(k_values, custom_metrics['davies_bouldin'], 'o-', label='Custom', linewidth=2, markersize=8)
        axes[0, 2].plot(k_values, sklearn_metrics['davies_bouldin'], 's-', label='Sklearn', linewidth=2, markersize=8)
        axes[0, 2].set_xlabel('Number of Clusters (K)', fontsize=10)
        axes[0, 2].set_ylabel('Davies-Bouldin Index', fontsize=10)
        axes[0, 2].set_title('Davies-Bouldin Index (Lower is Better)', fontweight='bold')
        axes[0, 2].legend()
        axes[0, 2].grid(alpha=0.3)

        # 4. Execution Time
        axes[1, 0].bar(np.array(k_values) - 0.2, custom_metrics['time'], width=0.4, label='Custom', alpha=0.8)
        axes[1, 0].bar(np.array(k_values) + 0.2, sklearn_metrics['time'], width=0.4, label='Sklearn', alpha=0.8)
        axes[1, 0].set_xlabel('Number of Clusters (K)', fontsize=10)
        axes[1, 0].set_ylabel('Time (seconds)', fontsize=10)
        axes[1, 0].set_title('Execution Time', fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(axis='y', alpha=0.3)

        # 5. Iterations to Converge
        axes[1, 1].bar(np.array(k_values) - 0.2, custom_metrics['iterations'], width=0.4, label='Custom', alpha=0.8)
        axes[1, 1].bar(np.array(k_values) + 0.2, sklearn_metrics['iterations'], width=0.4, label='Sklearn', alpha=0.8)
        axes[1, 1].set_xlabel('Number of Clusters (K)', fontsize=10)
        axes[1, 1].set_ylabel('Iterations', fontsize=10)
        axes[1, 1].set_title('Iterations to Convergence', fontweight='bold')
        axes[1, 1].legend()
        axes[1, 1].grid(axis='y', alpha=0.3)

        # 6. Cluster Visualization (PCA for 2D)
        if self.X.shape[1] > 2:
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(self.X)
        else:
            X_pca = self.X[:, :2]

        # Use optimal k (highest silhouette score)
        optimal_k = max(self.results.keys(),
                       key=lambda k: self.results[k]['custom']['silhouette'])

        custom_labels = self.results[optimal_k]['custom']['labels']

        scatter = axes[1, 2].scatter(X_pca[:, 0], X_pca[:, 1],
                                     c=custom_labels, cmap='viridis',
                                     alpha=0.6, s=30)
        axes[1, 2].set_xlabel('First Principal Component', fontsize=10)
        axes[1, 2].set_ylabel('Second Principal Component', fontsize=10)
        axes[1, 2].set_title(f'Cluster Visualization (K={optimal_k})', fontweight='bold')
        plt.colorbar(scatter, ax=axes[1, 2], label='Cluster')

        plt.tight_layout()
        plt.savefig('kmeans_comparison.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: kmeans_comparison.png")
        plt.close()

        # Save comparison table
        comparison_data = []
        for k in k_values:
            comparison_data.append({
                'K': k,
                'Method': 'Custom',
                'Inertia': self.results[k]['custom']['inertia'],
                'Silhouette': self.results[k]['custom']['silhouette'],
                'Davies-Bouldin': self.results[k]['custom']['davies_bouldin'],
                'Time (s)': self.results[k]['custom']['time'],
                'Iterations': self.results[k]['custom']['iterations']
            })
            comparison_data.append({
                'K': k,
                'Method': 'Sklearn',
                'Inertia': self.results[k]['sklearn']['inertia'],
                'Silhouette': self.results[k]['sklearn']['silhouette'],
                'Davies-Bouldin': self.results[k]['sklearn']['davies_bouldin'],
                'Time (s)': self.results[k]['sklearn']['time'],
                'Iterations': self.results[k]['sklearn']['iterations']
            })

        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_csv('kmeans_comparison_results.csv', index=False)
        print("✓ Saved: kmeans_comparison_results.csv")

        print("\n✓ All visualizations generated!")

        return comparison_df


# ============================================
# MAIN EXECUTION FUNCTION
# ============================================
def compare_kmeans_implementations(X, true_labels=None, k_values=[2, 3, 4, 5, 6],
                                   max_samples=10000, random_state=42):
    """
    Main function to compare custom and sklearn K-Means

    Parameters:
    -----------
    X : array-like or DataFrame
        Data to cluster (use preprocessed features, exclude lat/lon/class)
    true_labels : array-like, optional
        True class labels if available
    k_values : list
        List of k values to test
    max_samples : int
        Maximum number of samples to use (for memory efficiency)
    random_state : int
        Random seed for sampling

    Returns:
    --------
    comparator : KMeansComparison object
    results_df : DataFrame with comparison results
    """

    print("\n" + "="*70)
    print("K-MEANS CLUSTERING: IMPLEMENTATION COMPARISON")
    print("="*70)

    # Convert to DataFrame if needed
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    # Select only numeric columns
    print("\n[Data Preparation]")
    print("-" * 70)
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    X_numeric = X[numeric_cols].copy()

    print(f"Original features: {X.shape[1]}")
    print(f"Numeric features: {X_numeric.shape[1]}")
    print(f"Non-numeric features removed: {X.shape[1] - X_numeric.shape[1]}")
    print(f"Original samples: {X_numeric.shape[0]}")

    # Sample data if too large
    if X_numeric.shape[0] > max_samples:
        print(f"\n⚠ Dataset too large for memory! Sampling {max_samples} rows...")
        np.random.seed(random_state)
        sample_indices = np.random.choice(X_numeric.shape[0], max_samples, replace=False)
        X_numeric = X_numeric.iloc[sample_indices].reset_index(drop=True)
        if true_labels is not None:
            if isinstance(true_labels, pd.Series):
                true_labels = true_labels.iloc[sample_indices].reset_index(drop=True)
            else:
                true_labels = true_labels[sample_indices]
        print(f"✓ Sampled to {X_numeric.shape[0]} rows")

    # Remove columns with any missing values
    cols_before = X_numeric.shape[1]
    X_numeric = X_numeric.dropna(axis=1)
    if X_numeric.shape[1] < cols_before:
        print(f"Removed {cols_before - X_numeric.shape[1]} columns with NaN values")

    # Check for any remaining issues
    if X_numeric.isnull().sum().sum() > 0:
        print("Warning: Found NaN values, filling with column means...")
        X_numeric = X_numeric.fillna(X_numeric.mean())

    # Check for infinite values
    if np.isinf(X_numeric.values).sum() > 0:
        print("Warning: Found infinite values, replacing...")
        X_numeric = X_numeric.replace([np.inf, -np.inf], np.nan)
        X_numeric = X_numeric.fillna(X_numeric.mean())

    # Further reduce features if still too many
    if X_numeric.shape[1] > 50:
        print(f"\n⚠ Too many features ({X_numeric.shape[1]})! Selecting top 30 by variance...")
        variances = X_numeric.var()
        top_features = variances.nlargest(30).index
        X_numeric = X_numeric[top_features]
        print(f"✓ Reduced to {X_numeric.shape[1]} features")

    # Convert to float32 to save memory
    X_numeric = X_numeric.astype(np.float32)

    print(f"\n✓ Final dataset shape: {X_numeric.shape}")
    print(f"✓ Memory usage: {X_numeric.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"✓ Ready for clustering!")

    # Initialize comparator
    comparator = KMeansComparison(X_numeric.values, true_labels)

    # Run comparison
    comparator.run_comparison(k_values=k_values, random_state=random_state)

    # Generate visualizations
    results_df = comparator.plot_comparison()

    # Print summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    print("\nComparison Results:")
    print(results_df.to_string(index=False))

    # Determine optimal k
    best_k = max(comparator.results.keys(),
                 key=lambda k: comparator.results[k]['custom']['silhouette'])
    print(f"\n✓ Optimal K (based on Silhouette Score): {best_k}")

    return comparator, results_df


if __name__ == "__main__":
    print("\n" + "="*70)
    print("="*70)
    print("\n1. Load preprocessed data:")
    train_data = pd.read_csv('train_data_preprocessed.csv')
    print("\n2. Prepare features (automatically selects numeric columns):")
    X = train_data.drop('class', axis=1, errors='ignore')
    y = train_data['class'] if 'class' in train_data.columns else None
    print("\n3. Run comparison (with memory-efficient sampling):")
    comparator, results = compare_kmeans_implementations(
        X,
        true_labels=y,
        k_values=[2, 3, 4, 5],
        max_samples=500_000  # Adjust based on your RAM
    )